## 10.2 Vercor 버전 #1 : Vector2d 호환

In [40]:
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        # 보호된 객체 속성인 _components는 벡터 요소를 배열로 저장한다. 
        self._components = array(self.typecode, components)  

    def __iter__(self):
        return iter(self._components)  # 반복자 반환

    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    # 초구면좌표에 대한 공식을 이용하여 특정 좌표에 대한 각좌표 계산 함수
    def angle(self, n):  
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    # 각좌표를 모두 계산하는 제너레이터 표현식을 생성
    def angles(self):  
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        # 극좌표 대신 구면좌표(초구면좌표)를 사용
        if fmt_spec.endswith('h'):  
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  
            # 구면좌표 
            outer_fmt = '<{}>'  
        else:
            # 직교좌표
            coords = self
            outer_fmt = '({})'  
        components = (format(c, fmt_spec) for c in coords)  
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [3]:
Vector([3, 4, 5])

Vector([3.0, 4.0, 5.0])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 10.3 프로토콜과 덕 타이핑

In [5]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

## 10.4 Vector 버전 #2 : 슬라이스 가능한 시퀀스

In [7]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [8]:
v1[0], v1[-1]

(3.0, 5.0)

In [9]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

### 10.4.1 슬라이싱의 작동 방식

In [10]:
class MySeq:
    def __getitem__(self, index):
        return index

In [12]:
s = MySeq()
s[1]

1

In [14]:
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [15]:
slice

slice

In [16]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [17]:
slice(None, 10, 2).indices(5) # [0:5:2]와 동일하다

(0, 5, 2)

In [18]:
slice(-3, None, None).indices(5) # [2:5:1]과 동일하다. 

(2, 5, 1)

### 10.4.2 슬라이스를 인식하는 __getitem__()

In [20]:
def __len__(self):
    return len(self._components)

def __getitem__(self, index):
    cls = type(self)
    if isinstance(index, slice):
        return cls(self._components[index])
    elif isinstance(index, numbers.Integral):
        return self._components[index]
    else:
        msg = '{cls.__name__} indicies must be integers'
        raise TypeError(msg.format(cls=cls))

In [23]:
import numbers

v7 = Vector(range(7))
v7[-1]

6.0

In [24]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [30]:
v7[-1:]

Vector([6.0])

## 10.5 Vector 버전 #3: 동적 속성 접근

In [29]:
shorcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
        pos = cls.shorcut_names.find(name)
    if 0 <= pos < len(self._components):
        return self._components[pos]
    msg = f'{cls.__name__!r} object has no attribute {name!r}'
    raise AttributeError(msg)

In [33]:
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
        if name in cls.__match_args__:  
            error = 'readonly attribute {attr_name!r}'
        elif name.islower(): 
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''  
        if error:  
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
    super().__setattr__(name, value)  

## 10.6 Vector 버전 #4: 해싱 및 더 빠른 ==

In [34]:
n = 0
for i in range(6):
    n ^= i
n

1

In [35]:
import functools
functools.reduce(lambda a, b: a^b, range(6))

1

In [36]:
import operator
functools.reduce(operator.xor, range(6))

1

In [37]:
def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes)

In [38]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [39]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## 10.7 Vector 버전 #5: 포매팅

- 실수형 포맷 코드 : eEfFgGn%
- 정수형 포맷 코드 : bcdoxXn
- 문자열 : s
- 극좌표 : p
- 초평면좌표 : h

In [44]:
import itertools

In [45]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [46]:
format(Vector([2, 2, 2, 21]), '.3eh')

'<2.128e+01, 1.477e+00, 1.476e+00, 1.476e+00>'

In [47]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'

---

In [48]:
# 1차원
print(Vector([3.1, 4.2]))
print(Vector(range(10)))

(3.1, 4.2)
(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0)


In [49]:
# frombytes
v1_clone = Vector.frombytes(bytes(v1))
v1_clone, v1 == v1_clone

(Vector([3.0, 4.0, 5.0]), True)

In [50]:
# 다차원
v7 = Vector(range(7))
abs(v7)

9.539392014169456

In [51]:
# 시퀀스
v1 = Vector([3, 4, 5])
len(v1)

3

In [52]:
v1[0], v1[len(v1)-1], v1[-1]

(3.0, 5.0, 5.0)

In [53]:
# 슬라이싱
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [54]:
v7[1, 2]

TypeError: 'tuple' object cannot be interpreted as an integer

In [55]:
v7.t

3.0

In [56]:
v7.spam

AttributeError: 'Vector' object has no attribute 'spam'

---

In [57]:
# 직교좌표
v3 = Vector([3, 4, 5])
format(v3)

'(3.0, 4.0, 5.0)'

In [58]:
format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

In [59]:
# 구면좌표
print(format(Vector([1, 1]), 'h'))
print(format(Vector([1, 1]), '.3eh'))
print(format(Vector([1, 1]), '0.5fh'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>
